In [43]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import wordcloud
import re

## conda install -c conda-forge scrapy
from bs4 import BeautifulSoup 
#Beautiful Soup is used to parse and prettify raw web data
from collections import Counter
from wordcloud import WordCloud, STOPWORDS

In [44]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Get the data
url = "https://www.myrsten.nu/worldnet/beatlesongs.htm"
res = requests.get(url)
htmlData = res.content
#print(htmlData)

## Parse the Data
soup = BeautifulSoup(htmlData, "html.parser")

# Find all <td> elements
td_elements = soup.find_all("td")

# We want to store song, composer, and singer. 
songs = []

# Skip headers, then process title, composer, and singer
td_texts = [td.get_text(strip=True) for td in td_elements[4:]]  # Skip headers

for i in range(0, len(td_texts), 3):
    if i + 2 < len(td_texts):
        song_title = td_texts[i]
        composer = td_texts[i + 1]
        singer = td_texts[i + 2]
        
        song_title = re.sub(r'\"', ' ', song_title, flags = re.IGNORECASE)
        composer = re.sub(r'\"', ' ', composer, flags = re.IGNORECASE)
        singer = re.sub(r'\"', ' ', singer, flags = re.IGNORECASE)
        songs.append([song_title, composer, singer])

# # Write to csv file
# filename = "song_composer_singer.csv"

# # Open file
# my_file = open(filename, "w")

# # Write first rows, which are column names
# write_this = "Song, Composer, Singer \n"
# my_file.write(write_this)
# my_file.close()

# my_file = open(filename, "a")

for group in songs:
    song = group[0] 
    composer = group[1]
    singer = group[2]
    print("song: ", song, ", composer: ", composer, ", singer:", singer)

#     write_this = str(song) + "," + str(composer) + "," + str(singer) + "\n"
#     my_file.write(write_this)

# my_file.close()

#print(soup.prettify())

# #there are other parsers as well, like lxml or html5lib
# print(parsedData.prettify())

# texts = parsedData.findAll(text=True)
# #print(texts)

# #Getting the title of the HTML page.
# #print(parsedData.title)
# #print(parsedData.title.string)

# body = parsedData.find('body')
# # Extract the text
# text = body.get_text()
# print(text)
# #print(type(text))
# MyList=text.split()
# print(MyList)

# ##filter the list
# MyList = [item for item in MyList if item.isalpha()]
# print(MyList)
# MyList = [item for item in MyList if len(item) > 3]
# print(MyList)
# MyList=[x.lower() for x in MyList]
# print(MyList)
# StopWords=["website", "websites", "href", "this", "from"]
# MyList = [word for word in MyList if word not in StopWords]
# print(MyList)
# word_counts = Counter(MyList)
# print(word_counts)
# print(type(word_counts))

song:  Love Me Do , composer:  McCartney , singer: Lennon/McCartney
song:  P.S. I Love
You , composer:  McCartney , singer: McCartney
song:  Please
Please Me , composer:  Lennon , singer: Lennon/McCartney
song:  Ask Me Why , composer:  Lennon , singer: Lennon
song:  There´s
a Place , composer:  Lennon , singer: Lennon
song:  I Saw Her
Standing
There , composer:  McCartney , singer: McCartney
song:  Do You Want
to Know
a Secret , composer:  Lennon , singer: Harrison
song:  Misery , composer:  Lennon/McCartney , singer: Lennon/McCartney
song:  A Taste of
Honey , composer:  Scott/Marlow , singer: McCartney
song:  Anna (Go to
Him) , composer:  Alexander , singer: Lennon
song:  Boys , composer:  Dixon/Farrell , singer: Starr
song:  Chains , composer:  Goffin/King , singer: Harrison
song:  Baby
It´s You , composer:  David/Williams/Bacharach , singer: Lennon
song:  Twist and
Shout , composer:  Medley/Russel , singer: Lennon
song:  Total , composer:  Lennon:
4McCartney:
3L/M: 1 , singer: Lenno